# TEST 1: Leveraging LLMs for Feature Generation and Classification

Typically, if our data has $N$ features, we need around $10N$ data items to reach peak performance with classic classifiers like Logistic Regression. Therefore, if our vocabulary has 10,000 words, we would need around 1 million items in the training set to reach peak performance.

An interesting idea regarding this was explored in 2024 in [Balek, V., S'ykora, L., Sklen'ak, V., & Kliegr, T. (2024). LLM-based feature generation from text for interpretable machine learning. ArXiv, abs/2409.07132](https://arxiv.org/abs/2409.07132). The idea is to use an LLM to generate meaningful and interpretable features from text, and then use Logistic Regression for classification.

For example, in the movie plots dataset, we could have features like:
- "Is the protagonist an animal?" (0 or 1)
- "Does the plot indicate psychological suffering?" (0 or 1)

With a reasonable number of these features, our model could make predictions based on meaningful features instead of raw words.

## Objectives
* Perform feature extraction for a particular dataset
* Compare performance and explainability of classifiers with different approaches. 

## Rules

I highlight a few elements of our usual rules:

* You are **NOT ALLOWED** to use AI to generate any code you are asked to make yourself. This includes ChatGPT, CoPilot and all similar generators.
* You are **NOT ALLOWED** to use Google or any other search engine.
* You are **ALLOWED** to use the offical documentations for libraries: 
    * [sklearn](https://scikit-learn.org/)
    * [numpy](https://numpy.org/)
    * [matplotlib](https://matplotlib.org/)
    * [google AI studio](https://aistudio.google.com/)
* You are **ALLOWED** to use previous code from this course as basis.
* You **MUST** use the university's proctoring software to show you are complying with these rules
* This task is **INDIVIDUAL**. DO NOT share your code or results with anyone else.

## Tasks and Deliverables

* At any point, refer to [Balek et al.](https://arxiv.org/abs/2409.07132). 
* Make a well-commented code to solve each one of the tasks below.
* Each task will be evaluated as:
    * Insufficient: task is not done, off-topic, or low-effort
    * In process: task is incomplete, done with a clear conceptual error, or comments 
    * Proficient: everything works and comments are enough to understand what is being done
    * Advanced: everything works, comments are enough to understand what is being done, and code is well organized and formated using functions, dataclasses, and other adequate structures.
* This task should be finished by the end of the class.
* After you are finished, submit the executed notebook in our LMS system.

### 1. Dataset Preparation:
Adapting Balek et al.'s strategy to our movie plot classification case, create a dataset with at least 100 labeled items and at least 5 meaningful features. None of the features can be the class itself ("is this a drama plot?"). Use a clear strategy to avoid exceeding free tier quotas. Store data locally in a format of your choice.

### 2. Classification:
Use the generated features to train a Logistic Regression model. Use cross-validation to select the best hyperparameters. Report accuracy and f1-score for your classifier.

### 3. Performance Comparison
Compare the performance of the following approaches:
1. Traditional Bag-of-Words
2. LLM-generated features with Logistic Regression
3. Direct classification using LLM

Use a bar plot to show the performance differences (choose either accuracy or F1-score).

### 4. Improvement Strategies
Determine whether labeling more items would improve system performance. Use data to justify your answer.


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import classification_report

In [5]:
# {'classifier__C': 10.0, 'vectorizer__binary': True, 'vectorizer__max_df': 1.0, 'vectorizer__min_df': 1}

model_lr = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words='english')),
    ('classifier', LogisticRegression())
])

print(model_lr)

Pipeline(steps=[('vectorizer', TfidfVectorizer(stop_words='english')),
                ('classifier', LogisticRegression())])


In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/tiagoft/NLP/main/wiki_movie_plots_drama_comedy.csv')
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Plot'], df['Genre'], test_size=0.2)
# Train the pipeline
model_lr.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer(stop_words='english')),
                ('classifier', LogisticRegression())])

In [7]:
df.head()

,Plot,Genre
0,The film is about a family who move to the sub...,comedy
1,Before heading out to a baseball game at a nea...,comedy
2,The plot is that of a black woman going to the...,comedy
3,On a beautiful summer day a father and mother ...,drama
4,A thug accosts a girl as she leaves her workpl...,drama


In [4]:
y_pred = model_lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.77


In [5]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      comedy       0.77      0.62      0.69       845
       drama       0.77      0.87      0.82      1224

    accuracy                           0.77      2069
   macro avg       0.77      0.74      0.75      2069
weighted avg       0.77      0.77      0.76      2069



## 1. Is the wording descriptive language?

## 2. Does the text indicate suffering?

## 3. Do the characters seem goofy or funny?

## 4. Does the text use more complex wording?

## 5. Does the text leave unanswered questions?

In [2]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import time

load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
#GEMINI_API_KEY = # Go to https://aistudio.google.com/ to get a key. DO NOT commit your key to the repository!

# Start the use of the API
genai.configure(api_key=GEMINI_API_KEY)

In [14]:
# Make our prompt here
prompt_context = f"""
You will receive a text and must evaluate it based on five criteria. For each question, answer only with 0 (No) or 1 (Yes).

Evaluation Criteria:
Is the wording descriptive language?
Does the text indicate suffering?
Do the characters seem goofy or funny?
Does the text use more complex wording?
Does the text leave unanswered questions?
Format of Your Response:
Output your answers as a 5-character string of 0s and 1s, where:

The first character corresponds to question 1,
The second to question 2,
The third to question 3,
The fourth to question 4,
The fifth to question 5.
No spaces, no explanations, just the 5-character string (example: 10010)."""

generation_config = genai.GenerationConfig(
    max_output_tokens=5,
)

# Use our prompt four times
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

df_processed = pd.DataFrame(columns=["Plot", "Q1", "Q2", "Q3", "Q4", "Q5", "Genre"])

i = 0

while i < 100:
    try:
        text = df['Plot'].iloc[i]
        genre = df['Genre'].iloc[i]

        response = model.generate_content([prompt_context, text],
                                        generation_config=generation_config)
        
        print(response.text)

        response = response.text
        
        df_processed = pd.concat([df_processed, pd.DataFrame({'Plot': [text], 'Q1': [response[0]], 'Q2': [response[1]], 'Q3': [response[2]], 'Q4': [response[3]], 'Q5': [response[4]], 'Genre': [genre]})])

        i += 1
    except Exception as e:
        print(e)
        time.sleep(62)

df_processed.to_csv('movies_processed.csv', index=False)
        

11001
01001
01001
01001
01001
01001
11010
11001
11010
11001
01001
01001
01001
11001
11011
429 Resource has been exhausted (e.g. check quota).
11011
01000
00011
11001
01001
00100
01000
11001
01001
01001
01001
11001
11011
11011
11001
11001
11001
429 Resource has been exhausted (e.g. check quota).
01000
11101
11011
01001
11011
01001
11001
11011
01001
11001
11011
00001
11001
11001
01001
429 Resource has been exhausted (e.g. check quota).
01001
01000
11001
01001
01001
01001
11001
11011
01001
11011
01001
11001
01001
00000
00001
01001
01001
429 Resource has been exhausted (e.g. check quota).
01001
11001
11001
11001
11001
01001
11011
01001
01000
01001
01001
11001
11011
11001
01001
11001
00001
01001
11001
01001
01001
01000
01001
11101
11001
11001
429 Resource has been exhausted (e.g. check quota).
01001
11011
11001
01001
01001
11011
01000
11001
11001
01001


In [18]:
# Transform the question cols into ints
df_processed['Q1'] = df_processed['Q1'].astype(int)
df_processed['Q2'] = df_processed['Q2'].astype(int)
df_processed['Q3'] = df_processed['Q3'].astype(int)
df_processed['Q4'] = df_processed['Q4'].astype(int)
df_processed['Q5'] = df_processed['Q5'].astype(int)

In [30]:
# for each Q column, I want to know how many are 1

print(df_processed["Q1"].value_counts())
print(df_processed["Q2"].value_counts())
print(df_processed["Q3"].value_counts())
print(df_processed["Q4"].value_counts())
print(df_processed["Q5"].value_counts())

Q1
0    52
1    48
Name: count, dtype: int64
Q2
1    94
0     6
Name: count, dtype: int64
Q3
0    97
1     3
Name: count, dtype: int64
Q4
0    83
1    17
Name: count, dtype: int64
Q5
1    89
0    11
Name: count, dtype: int64


In [24]:
model_lr = Pipeline([
    ('classifier', LogisticRegression())
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_processed[["Q1", "Q2", "Q3", "Q4", "Q5"]], df_processed['Genre'], test_size=0.2)

# Train the pipeline
model_lr.fit(X_train, y_train)

Pipeline(steps=[('classifier', LogisticRegression())])

In [25]:
y_pred = model_lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.55


In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      comedy       1.00      0.18      0.31        11
       drama       0.50      1.00      0.67         9

    accuracy                           0.55        20
   macro avg       0.75      0.59      0.49        20
weighted avg       0.78      0.55      0.47        20



# **TIAGO VERSION**

In [11]:
import json

# Make our prompt here
prompt_context = f"""
You will receive a movie plot and your task is to answer 5 questions about it. For each question, answer only with 0 (No) or 1 (Yes). If you are in doubt,
answer with the option that seems most likely to you.

The format for the answer should be a JSON object with the following structure:
{{
    "Q1": "Answer to Question 1",
    "Q2": "Answer to Question 2",
    "Q3": "Answer to Question 3",
    "Q4": "Answer to Question 4",
    "Q5": "Answer to Question 5"
}}

Questions:
1. Does the plot involve psychologial or mental suffering?
2. Will this movie make me rethink my life choices?
3. Am I going to cry when I watch this movie?
4. Is the plot absurd or exaggerated?
5. Could the situations in the movie realistically happen to the average person?

The movie plot is:
"""

generation_config = genai.GenerationConfig(
    max_output_tokens=100,
    temperature=0.0,
    response_mime_type="application/json"
)

# Use our prompt four times
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

df_processed = pd.DataFrame(columns=["Plot", "Q1", "Q2", "Q3", "Q4", "Q5", "Genre"])

i = 0

while i < 100:
    try:
        text = df['Plot'].iloc[i]
        genre = df['Genre'].iloc[i]

        response = model.generate_content([prompt_context, text],
                                        generation_config=generation_config)

        response = json.loads(response.text)

        print(response)
        
        df_processed = pd.concat([df_processed, pd.DataFrame({'Plot': [text], 'Q1': [response["Q1"]], 'Q2': [response["Q2"]], 'Q3': [response["Q3"]], 'Q4': [response["Q4"]], 'Q5': [response["Q5"]], 'Genre': [genre]})])

        i += 1
    except Exception as e:
        print(e)
        time.sleep(62)

df_processed.to_csv('movies_processed_tiago.csv', index=False)
        

{'Q1': '1', 'Q2': '0', 'Q3': '0', 'Q4': '0', 'Q5': '1'}
{'Q1': '0', 'Q2': '0', 'Q3': '0', 'Q4': '1', 'Q5': '1'}
{'Q1': '0', 'Q2': '0', 'Q3': '0', 'Q4': '1', 'Q5': '0'}
{'Q1': '0', 'Q2': '0', 'Q3': '1', 'Q4': '0', 'Q5': '0'}
{'Q1': '1', 'Q2': '0', 'Q3': '1', 'Q4': '0', 'Q5': '0'}
{'Q1': '0', 'Q2': '0', 'Q3': '0', 'Q4': '0', 'Q5': '1'}
{'Q1': '1', 'Q2': '1', 'Q3': '0', 'Q4': '0', 'Q5': '0'}
{'Q1': '1', 'Q2': '0', 'Q3': '1', 'Q4': '0', 'Q5': '0'}
{'Q1': '1', 'Q2': '0', 'Q3': '0', 'Q4': '0', 'Q5': '1'}
{'Q1': '1', 'Q2': '0', 'Q3': '1', 'Q4': '0', 'Q5': '0'}
{'Q1': '1', 'Q2': '0', 'Q3': '0', 'Q4': '0', 'Q5': '1'}
{'Q1': '0', 'Q2': '0', 'Q3': '0', 'Q4': '1', 'Q5': '0'}
{'Q1': '1', 'Q2': '1', 'Q3': '1', 'Q4': '0', 'Q5': '0'}
{'Q1': '1', 'Q2': '0', 'Q3': '0', 'Q4': '0', 'Q5': '0'}
{'Q1': '1', 'Q2': '0', 'Q3': '1', 'Q4': '0', 'Q5': '0'}
{'Q1': '1', 'Q2': '0', 'Q3': '0', 'Q4': '0', 'Q5': '0'}
{'Q1': '1', 'Q2': '0', 'Q3': '0', 'Q4': '0', 'Q5': '0'}
429 Resource has been exhausted (e.g. check quot

In [12]:
# Transform the question cols into ints
df_processed['Q1'] = df_processed['Q1'].astype(int)
df_processed['Q2'] = df_processed['Q2'].astype(int)
df_processed['Q3'] = df_processed['Q3'].astype(int)
df_processed['Q4'] = df_processed['Q4'].astype(int)
df_processed['Q5'] = df_processed['Q5'].astype(int)

In [31]:
model_lr = Pipeline([
    ('classifier', LogisticRegression())
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_processed[["Q1", "Q2", "Q3", "Q4", "Q5"]], df_processed['Genre'], test_size=0.2)

# Train the pipeline
model_lr.fit(X_train, y_train)

Pipeline(steps=[('classifier', LogisticRegression())])

In [32]:
y_pred = model_lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.90


In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      comedy       0.75      1.00      0.86         6
       drama       1.00      0.86      0.92        14

    accuracy                           0.90        20
   macro avg       0.88      0.93      0.89        20
weighted avg       0.93      0.90      0.90        20

